globals [
  initial-trees   ;; how many trees (green patches) we started with
  burned-trees    ;; how many have burned so far
]

breed [fires fire]    ;; bright red turtles -- the leading edge of the fire
breed [embers ember]  ;; turtles gradually fading from red to near black

to setup
  clear-all
  set-default-shape turtles "square"
  ;; make some green trees
  ask patches with [(random-float 100) < density]
    [ set pcolor green ]
  ;; make a column of burning trees
  ask patches with [pxcor = min-pxcor]
    [ ignite ]
  ;; set tree counts
  set initial-trees count patches with [pcolor = green]
  set burned-trees 0
  reset-ticks
end

to go
  if not any? turtles  ;; either fires or embers
    [ stop ]
  ask fires
    [ ask neighbors4 with [pcolor = green]
        [ ignite ]
      set breed embers ]
  fade-embers
  tick
end

;; creates the fire turtles
to ignite  ;; patch procedure
  sprout-fires 1
    [ set color red ]
  set pcolor black
  set burned-trees burned-trees + 1
end

;; achieve fading color effect for the fire as it burns
to fade-embers
  ask embers
    [ set color color - 0.3  ;; make red darker
      if color < red - 3.5     ;; are we almost at black?
        [ set pcolor color
          die ] ]
end


The MESA equivalent for the NetLogo model is given below:

```python
import mesa
from mesa import Model
from mesa.space import SingleGrid
from mesa import Agent
import random


class TreeCell(Agent):
    """
    A tree cell.

    Attributes:
        x, y: Grid coordinates
        condition: Can be "Fine", "On Fire", or "Burned Out"
    """

    def __init__(self, model, pos):
        """
        Create a new tree.
        Args:
            pos: The tree's coordinates on the grid. Used by the grid
            to place the agent.
        """
        super().__init__(pos, model)
        self.pos = pos
        self.condition = "Fine"

    def step(self):
        """
        If the tree is on fire, spread it to fine trees nearby.
        """
        if self.condition == "On Fire":
            neighbors = self.model.grid.get_neighbors(self.pos, moore=False)
            for neighbor in neighbors:
                if neighbor.condition == "Fine":
                    neighbor.condition = "On Fire"
            self.condition = "Burned Out"


class ForestFire(Model):
    """
    Simple Forest Fire model.
    """

    def __init__(self, height=100, width=100, density=0.65):
        """
        Create a new forest fire model.

        Args:
            height, width: The size of the grid to model
            density: What fraction of grid cells have a tree in them.
        """
        # Initialize model parameters
        self.height = height
        self.width = width
        self.density = density

        # Set up model objects
        self.schedule = mesa.time.RandomActivation(self)
        self.grid = SingleGrid(width, height, torus=False)

        # Place a tree in each cell with Prob = density
        for (contents, x, y) in self.grid.coord_iter():
            if random.random() < self.density:
                # Create a tree
                new_tree = TreeCell(self, (x, y))
                # Set all trees in the first column on fire.
                if x == 0:
                    new_tree.condition = "On Fire"
                self.grid._place_agent((x, y), new_tree)
                self.schedule.add(new_tree)

    def step(self):
        """
        Advance the model by one step.
        """
        self.schedule.step()


# Create and run the model
model = ForestFire(100, 100)
for i in range(0, 100):
    model.step()
```

In this Python code, we create a simple forest fire model by using the MESA framework. We represent each cell on a grid as a tree. At each step, if a tree is on fire, it can spread the fire to any adjacent tree (4-neighbor grid: north, south, east, west) that is not already burned down or on fire. After spreading its fires (if any), the tree burns out and can no longer spread fire.

Please, make sure to install Mesa with `pip install mesa` before running the code.

In [2]:
from abses import MainModel, Actor


class Tree(Actor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.condition = "green"

    def step(self):
        if self.condition == "burning":
            # TODO: Write code to spread the fire
            # Spread the fire to nearby trees
            for neighbor in self.iter_neighbors(pos=self.pos, moore=False):
                pass
            # Become burned down


class Forest(MainModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # Initialize a grid
        self.nature.create_module(how="from_resolution", shape=(4, 4))

        # Create initial agents
        self.agents.create(Tree, 10)

        # TODO: Position the agents across the grid

    def step(self):
        self.actors.trigger("step")

In [3]:
parameters = {
    "time": {
        "end": 100,
    }
}


model = Forest(parameters=parameters)

model.run_model()

[2024-01-20 22:31:08][human          ] | Initializing a new Human Module...
[2024-01-20 22:31:08][nature         ] | Initializing a new Base Nature module...
[2024-01-20 22:31:09][nature         ] | Initializing a new Model Layer...
[2024-01-20 22:31:09][nature         ] | Using rioxarray version: 0.13.4
[2024-01-20 22:31:09][container      ] | Created 10 actors of breed Tree
[2024-01-20 22:31:09][main           ] | Setting up Forest...
[2024-01-20 22:31:09][main           ] | Ending Forest
